In [1]:
import sys
sys.path.insert(0, 'D:\\StuDY\\2023 interview prepare')
from db_credentials import *
import mysql.connector
from mysql.connector import Error
import boto3
import json
import os
import pandas as pd

In [ ]:
try:
    # Connect to the database
    connection = mysql.connector.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = connection.cursor()
except Error as e:
    print(f"Error connecting to MySQL database: {e}")


# Set up S3 client
s3 = boto3.client('s3', aws_access_key_id = ACCESS_KEY_ID,  aws_secret_access_key = SECRET_ACCESS_KEY)

# Specify S3 bucket name and folder prefix
bucket_name = Bucket_Name
folder_prefix = Folder_Prefix

files = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

file_list = []
object_list = []
for file in files['Contents']:
    folder_name = file['Key'].split('/')[-1]
    if len(folder_name) !=0:
        s3_object = s3.get_object(Bucket=Bucket_Name, Key=file['Key'])
        file_content = s3_object['Body'].read().decode('utf-8')
        jsonObject = json.loads(file_content)       
        df = pd.DataFrame.from_dict(jsonObject) 
        file_list.append(folder_name)
        for index, row in df.iterrows():
            sql = "INSERT ignore INTO Albums (album_id, album_name, album_type, album_tracks, album_url, album_release_date) VALUES (%s, %s, %s, %s, %s, %s)"
            val = (row['album_id'], row['album_name'], row['album_type'], row['album_tracks'], row['album_url'], row['album_release_date'])
            cursor.execute(sql, val)
            connection.commit()

            # SQL query to insert a new record into the Artists table
            sql = "INSERT ignore INTO Artists (artist_id, artist_name, artist_url, artist_genre, artist_followers, artist_popularity, record_date) VALUES (%s, %s, %s, %s, %s, %s)"
            val = (row['artist_id'], row['artist_name'], row['artist_url'], ','.join(row['artist_genre']), row['artist_followers'], row['artist_popularity'], row['record_date'])
            cursor.execute(sql, val)
            connection.commit()

            # SQL query to insert a new record into the Tracks table
            sql = "INSERT INTO Tracks (track_id, album_id, artist_id, record_date, track_url, track_popularity, danceability, energy, key_, loudness, mode_, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms, time_signature) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            val = (row['track_id'], row['album_id'], row['artist_id'], row['record_date'], row['track_url'], row['track_popularity'], row['danceability'], row['energy'], row['key'], row['loudness'], row['mode'], row['speechiness'], row['acousticness'], row['instrumentalness'], row['liveness'], row['valence'], row['tempo'], row['duration_ms'], row['time_signature'])
            cursor.execute(sql, val)
            connection.commit()

In [ ]:
# Close the database connection
if connection.is_connected():
    cursor.close()
    connection.close()
    print("MySQL connection closed")